**Machine Learning Assignment 4**

Content:

1.   Cuda Check
2.   Install Plug in to let write and execute cuda code in google colab
3.   Cuda Code
4.   Cuda Output
5.   **Python Codes**

    5.a. Python code for *Assignment 3 part 2*

    5.b. Python Code for *Assignment 4* 

1. Check Cuda

In [67]:

!/opt/bin/nvidia-smi

Thu Nov 25 23:49:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

2. Install Plug in

In [68]:

!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-7c0sl0pl
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-7c0sl0pl
The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


3. Cuda Code for Assignment 4 with improvement from the previous code (Assignment 3 Part 2)

In [69]:
%%cuda --name matrix_cublas_v2.cu 
// !nvcc -o /content/src/matrix_cublas_v2 /content/src/matrix_cublas_v2.cu -lcublas -lcurand

#include <stdio.h>
#include <cublas_v2.h>
#include <time.h>
#include "cuda_runtime.h"
#include "curand.h"


#define CUDA_CALL(x) do { if ((x) != cudaSuccess) { \
    printf("Error at %s:%d\n", __FILE__, __LINE__);\
    return EXIT_FAILURE; }} while(0)


cudaError_t checkCuda();


/**
 * Run the matrix multiplication using CUDA cuBlas
 */
int matrixMultiply(dim3& dimsA, dim3& dimsB, int N)
{
    // Allocate host and device memory for matrices A, B and C
    unsigned int size_A = dimsA.x * dimsA.y;
    unsigned int mem_size_A = sizeof(float) * size_A;
    float* h_A = NULL;
    CUDA_CALL(cudaHostAlloc(&h_A, mem_size_A, cudaHostAllocDefault));
    float* d_A = NULL;
    CUDA_CALL(cudaMalloc(&d_A, mem_size_A));

    unsigned int size_B = dimsB.x * dimsB.y;
    unsigned int mem_size_B = sizeof(float) * size_B;
    float* h_B = NULL;
    CUDA_CALL(cudaHostAlloc(&h_B, mem_size_B, cudaHostAllocDefault));
    float* d_B = NULL;
    CUDA_CALL(cudaMalloc(&d_B, mem_size_B));

    dim3 dimsC(dimsB.x, dimsA.y, 1);
    unsigned int size_C = dimsC.x * dimsC.y;
    unsigned int mem_size_C = sizeof(float) * size_C;
    float *h_C = NULL;
    CUDA_CALL(cudaHostAlloc(&h_C, mem_size_C, cudaHostAllocDefault));
    float *d_C = NULL;
    CUDA_CALL(cudaMalloc(&d_C, mem_size_C));

    // initiate the random generator on GPU
    curandGenerator_t generator;
    CUDA_CALL(curandCreateGenerator(&generator, CURAND_RNG_PSEUDO_XORWOW));
    CUDA_CALL(curandSetPseudoRandomGeneratorSeed(generator, (int)time(NULL)));

    // create cuda stream
    const int NUM_STREAMS = 2;
    cudaStream_t streams[NUM_STREAMS];
    for (int i = 0; i < NUM_STREAMS; ++i)
    {
        CUDA_CALL(cudaStreamCreate(&streams[i]));
    }

    // Allocate CUDA events that we'll use for timing
    cudaEvent_t start, stop;
    CUDA_CALL(cudaEventCreate(&start));
    CUDA_CALL(cudaEventCreate(&stop));

    // create cublas handle
    const float alpha = 1.0f;
    const float beta  = 0.0f;
    cublasHandle_t handle;
    CUDA_CALL(cublasCreate(&handle));

    // Starting
    CUDA_CALL(cudaEventRecord(start, NULL));
    for(int i = 0; i < N; ++i)
    {
        // Initialising streams
        int stream_index = i % NUM_STREAMS;
        cudaStream_t stream = streams[stream_index];

        // generate device matrixes d_A and d_B directly
        CUDA_CALL(curandGenerateUniform(generator, d_A, size_A));
        CUDA_CALL(curandGenerateUniform(generator, d_B, size_B));

        // caculate matrix C = A * B by using cuBlas
        cublasSetStream(handle, stream);
        CUDA_CALL(cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N, dimsB.x, dimsA.y, dimsA.x, &alpha, 
            d_B, dimsB.x, d_A, dimsA.x, &beta, d_C, dimsB.x));

        // copy result from device to host
        CUDA_CALL(cudaMemcpyAsync(h_C, d_C, mem_size_C, cudaMemcpyDeviceToHost, stream));
    }

    for (int i = 0; i < NUM_STREAMS; ++i)
    {
        CUDA_CALL(cudaStreamSynchronize(streams[i]));
    }

    // Record the stop event
    CUDA_CALL(cudaEventRecord(stop, NULL));
    CUDA_CALL(cudaEventSynchronize(stop));

    float msec_TotalMatrixMul = 0.0f;
    CUDA_CALL(cudaEventElapsedTime(&msec_TotalMatrixMul, start, stop));

    // Compute and print the performance
    float msec_MatrixMul = msec_TotalMatrixMul / N;
    double flops_MatrixMul = 2.0 * (double)dimsA.x * (double)dimsA.y * (double)dimsB.x;
    double flopsGiga = (flops_MatrixMul * 1.0e-9f) / (msec_MatrixMul / 1000.0f);
    printf("Performance= %.2f GFlop/s, AVGTime= %.3f msec, TotalTime=%.3f msc \n",
        flopsGiga, msec_MatrixMul, msec_TotalMatrixMul);
 
    // Memory Clean up
    cudaFreeHost(h_A);
    cudaFreeHost(h_B);
    cudaFreeHost(h_C);
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    for (int i = 0; i < NUM_STREAMS; ++i)
    {
        CUDA_CALL(cudaStreamDestroy(streams[i]));
    }

    return EXIT_SUCCESS;
} 

int main(int argc, char** argv)
{
    if(checkCuda() != cudaSuccess)
    {
        return 0;
    }
 
    // condition i) A(500*500), B(500*400), N=100
    printf("Performing condition i: A(500*500), B(500*400), N=100\n");
    int N1 = 100;
    dim3 dimsA1(500, 500, 1);
    dim3 dimsB1(400, 500, 1);
    matrixMultiply(dimsA1, dimsB1, N1);
    printf("\n");
 
    // condition ii) A(50*20), B(20*50), N=5000
    printf("Performing condition ii: A(50*20), B(20*50), N=5000\n");
    int N2 = 5000;
    dim3 dimsA2(20, 50, 1);
    dim3 dimsB2(50, 20, 1);
    matrixMultiply(dimsA2, dimsB2, N2);
    printf("\n");
 
    // condition iii) A(6*4000), B(4000*9), N=1000
    printf("Performing condition iii: A(6*4000), B(4000*9), N=1000\n");
    int N3 = 1000;
    dim3 dimsA3(4000, 6, 1);
    dim3 dimsB3(9, 4000, 1);
    matrixMultiply(dimsA3, dimsB3, N3);
    printf("\n");

    return 0;
}

cudaError_t checkCuda()
{
    printf("Checking CUDA...\n");

    int devID = 0;
    cudaError_t error;
    cudaDeviceProp deviceProp;
    error = cudaGetDevice(&devID);

    if (error != cudaSuccess)
    {
        printf("cudaGetDevice returned error %s (code %d), line(%d)\n", cudaGetErrorString(error), error, __LINE__);
        return error;
    }

    error = cudaGetDeviceProperties(&deviceProp, devID);

    if (deviceProp.computeMode == cudaComputeModeProhibited)
    {
        fprintf(stderr, "Error: device is running in <Compute Mode Prohibited>, no threads can use ::cudaSetDevice().\n");
        exit(EXIT_SUCCESS);
    }

    if (error != cudaSuccess)
    {
        printf("cudaGetDeviceProperties returned error %s (code %d), line(%d)\n", cudaGetErrorString(error), error, __LINE__);
    }
    else
    {
        printf("GPU Device %d: \"%s\" with compute capability %d.%d\n\n", devID, deviceProp.name, deviceProp.major, deviceProp.minor);
    }

    return error;
}

'File written in /content/src/matrix_cublas_v2.cu'

In [70]:
!nvcc -o /content/src/matrix_cublas_v2 /content/src/matrix_cublas_v2.cu -lcublas -lcurand

/content/src/matrix_cublas_v2.cu: In function ‘int matrixMultiply(dim3&, dim3&, int)’:
/content/src/matrix_cublas_v2.cu:48:87: warning: comparison between ‘curandStatus_t {aka enum curandStatus}’ and ‘enum cudaError’ [-Wenum-compare]
     CUDA_CALL(curandCreateGenerator(&generator, CURAND_RNG_PSEUDO_XORWOW));
                                                                                       ^
/content/src/matrix_cublas_v2.cu:49:92: warning: comparison between ‘curandStatus_t {aka enum curandStatus}’ and ‘enum cudaError’ [-Wenum-compare]
     CUDA_CALL(curandSetPseudoRandomGeneratorSeed(generator, (int)time(NULL)));
                                                                                            ^
/content/src/matrix_cublas_v2.cu:68:52: warning: comparison between ‘enum cublasStatus_t’ and ‘enum cudaError’ [-Wenum-compare]
     CUDA_CALL(cublasCreate(&handle));
                                                    ^
/content/src/matrix_cublas_v2.cu:79:73: warning: compariso

4. Cuda Output

In [71]:
!/content/src/matrix_cublas_v2

Checking CUDA...
GPU Device 0: "Tesla K80" with compute capability 3.7

Performing condition i: A(500*500), B(500*400), N=100
Performance= 344.20 GFlop/s, AVGTime= 0.581 msec, TotalTime=58.105 msc 

Performing condition ii: A(50*20), B(20*50), N=5000
Performance= 1.02 GFlop/s, AVGTime= 0.098 msec, TotalTime=490.836 msc 

Performing condition iii: A(6*4000), B(4000*9), N=1000
Performance= 2.17 GFlop/s, AVGTime= 0.199 msec, TotalTime=198.718 msc 



**Python Code**

5. a. Python Version for Assignment 3 Part 2

In [72]:
# #Python code for Assignment 3 Part 2

# import numpy as np 
# from timeit import default_timer as timer
# import threading

# # utilising threads
# class cudamatrixthread(threading.Thread):
    
#     def __init__(self, s_A, B, s_N):
#         super(cudamatrixthread, self).__init__()
#         self.s_A = s_A
#         self.B = B
#         self.s_N = s_N

#         self.Cs = []

#     def run(self) -> None:
#         for _ in range(self.s_N):
#             s = np.random.rand(self.s_A[0], self.s_A[1])
#             d = np.matmul(s, self.B)
#             self.Cs.append(d)

# # function to perform multiplication
# def cuda_multiplication(s_1, s_2, N):
#     hight_z, width_z = s_1
#     hight_x, width_x = s_2
#     B = np.random.rand(hight_x, width_x)
    
#     tot_thread = 10
#     threads = []
#     s_N = N // tot_thread
#     s_N_last = N % tot_thread
    
#     start = timer()
#     for i in range(tot_thread + 1):
#         t_N = s_N
#         if s_N_last > 0 and i == tot_thread:
#             t_N = s_N_last

#         thread = cudamatrixthread(s_1, B, t_N)
#         threads.append(thread)
#         thread.start()

#     # waiting all thread finish
#     for thread in threads:
#         thread.join()
    
#     # get result from each thread
#     for thread in threads:
#         for C in thread.Cs:
#             ret_C = C
#             # print(C.shape)

#     end = timer()

#     matricmulti = (end - start) * 1000
#     matrix_m = matricmulti / N
#     f_matricmul = 2.0 * width_z * hight_z * width_x
#     p_gig = (f_matricmul * 1.0e-9) / (matrix_m / 1000.0)
#     print("Performance= {:.2f} GFlop/s, AVGTime= {:.3f} msec, TotalTime= {:.3f} msc \n"
#             .format(p_gig, matrix_m, matricmulti))

# def main():
#     # condition i) A(500*500), B(500*400), N=100
#     print("Performing condition i: A(500*500), B(500*400), N=100")
#     cuda_multiplication((500, 500), (500, 400), 100)

#     # condition ii) A(50*20), B(20*50), N=5000
#     print("Performing condition ii: A(50*20), B(20*50), N=5000")
#     cuda_multiplication((50, 20), (20, 50), 5000)

#     # condition iii) A(6*4000), B(4000*9), N=1000
#     print("Performing condition iii: A(6*4000), B(4000*9), N=1000")
#     cuda_multiplication((6, 4000), (4000, 9), 1000)

# if __name__ == '__main__':
#     main()


5. b. Python code for assignment 4 

In [73]:
#Python code for Assignment 4


import numpy as np 
from timeit import default_timer as timer
import threading


class cudamatrixthread(threading.Thread):
    
    def __init__(self, s_A, s_B, s_N):
        super(cudamatrixthread, self).__init__()
        self.s_A = s_A
        self.s_B = s_B
        self.s_N = s_N

        self.Cs = []

    def run(self) -> None:
        for _ in range(self.s_N):
            x = np.random.rand(self.s_A[0], self.s_A[1])
            c = np.random.rand(self.s_B[0], self.s_B[1])
            z = np.matmul(x, c)
            self.Cs.append(z)


def cuda_multiplication(s_1, s_2, N):
    hight_z, width_z = s_1
    hight_x, width_x = s_2
    B = np.random.rand(hight_x, width_x)
    
    tot_thread = 10
    threads = []
    s_N = N // tot_thread
    s_N_last = N % tot_thread
    
    start = timer()
    for i in range(tot_thread + 1):
        t_N = s_N
        if s_N_last > 0 and i == tot_thread:
            t_N = s_N_last

        thread = cudamatrixthread(s_1, s_2, t_N)
        threads.append(thread)
        thread.start()

    # waiting all thread finish
    for thread in threads:
        thread.join()
    
    # get result from each thread
    for thread in threads:
        for C in thread.Cs:
            ret_C = C
            # print(C.shape)

    end = timer()

    matricmulti = (end - start) * 1000
    matrix_m = matricmulti / N
    f_matricmul = 2.0 * width_z * hight_z * width_x
    p_gig = (f_matricmul * 1.0e-9) / (matrix_m / 1000.0)
    print("Performance= {:.2f} GFlop/s, AVGTime= {:.3f} msec, TotalTime= {:.3f} msc \n"
            .format(p_gig, matrix_m, matricmulti))

def main():
    # condition i) A(500*500), B(500*400), N=100
    print("Performing condition i: A(500*500), B(500*400), N=100")
    cuda_multiplication((500, 500), (500, 400), 100)

    # condition ii) A(50*20), B(20*50), N=5000
    print("Performing condition ii: A(50*20), B(20*50), N=5000")
    cuda_multiplication((50, 20), (20, 50), 5000)

    # condition iii) A(6*4000), B(4000*9), N=1000
    print("Performing condition iii: A(6*4000), B(4000*9), N=1000")
    cuda_multiplication((6, 4000), (4000, 9), 1000)

if __name__ == '__main__':
    main()


Performing condition i: A(500*500), B(500*400), N=100
Performance= 13.07 GFlop/s, AVGTime= 15.304 msec, TotalTime= 1530.447 msc 

Performing condition ii: A(50*20), B(20*50), N=5000
Performance= 1.03 GFlop/s, AVGTime= 0.097 msec, TotalTime= 486.003 msc 

Performing condition iii: A(6*4000), B(4000*9), N=1000
Performance= 0.49 GFlop/s, AVGTime= 0.890 msec, TotalTime= 890.124 msc 

